# 1. Librerías a implementar

In [1]:
#Librería para comando de sistema
import os

#Librerías para manejo de datos
import pandas as pd
import numpy as np

#Librerías de machine learning
#Para realizar la separación del conjunto de aprendizaje en entrenamiento y test
from sklearn.model_selection import train_test_split
#Para construir un modelo con el algoritmo de regresión lineal
from sklearn.linear_model import LinearRegression
#Para determinar el rendimiento del modelo con las métricas MSE, MAE y R2
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#Librería para parámetros polinomiales
from sklearn.preprocessing import PolynomialFeatures

#Para la creación del modelo Ridge
from sklearn.linear_model import Ridge
#Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
#Par la validación cruzada
from sklearn.model_selection import KFold

#Para la creación del modelo Lasso
from sklearn.linear_model import Lasso

#Para sacar un reporte estadístico que podemos usar para determinar la importancia de las variables explicativas
import statsmodels.api as sm

# 2. Carga de los datos
Se carga el set de datos que se encuentra disponible

In [2]:
#Se cargan los datos
df = pd.read_excel('venta_inmuebles_data.xlsx', sheet_name='in')

In [3]:
#Mostrar datos
df.head()

,numero_cuartos,numero_baños,area_de_ construcción_pie2,area_del lote_pie2,numero_pisos,condición,grado,superficie_sótano_pie2,año_de_construcción,precio
0,3,1,1180,5650,1,3,7,0,1955,221900
1,2,1,770,10000,1,3,6,0,1933,180000
2,4,3,1960,5000,1,5,7,910,1965,604000
3,3,2,1680,8080,1,3,8,0,1987,510000
4,3,1,1780,7470,1,3,7,730,1960,229500


In [4]:
#Cantidad de datos y número de variables
df.shape

(5466, 10)

# 3. Preparación de los datos
Antes de realizar cualquier paso de preparación es necesario asegurar de limpiar los dato como aquellos faltantes y los registros duplicados.

In [5]:
#Es recomendable que todos los pasos de preparación se realicen en otro dataframe
df_t = df

In [6]:
#Se visualiza los registros nulos a continuación:
df_t.isna().sum()/len(df_t)

numero_cuartos                0.0
numero_baños                  0.0
area_de_ construcción_pie2    0.0
area_del lote_pie2            0.0
numero_pisos                  0.0
condición                     0.0
grado                         0.0
superficie_sótano_pie2        0.0
año_de_construcción           0.0
precio                        0.0
dtype: float64

Teniendo en cuenta lo anterior observamos que no hay registros nulos

In [7]:
#Se explora el dataframe para saber si hay registros duplicados
df_t = df_t.drop_duplicates()

In [8]:
#Se observa el tamaño del df para comprobar si se eliminaro registros duplicados
df_t.shape

(5465, 10)

In [9]:
#Se muestran las categorías de la variable "numero_cuartos"
pd.value_counts(df["numero_cuartos"])

3     2559
2     1653
4      799
5      236
1      132
6       64
7        8
0        6
8        4
9        2
10       2
11       1
Name: numero_cuartos, dtype: int64

In [10]:
#Se muestran las categorías de la variable "numero_baños"
pd.value_counts(df["numero_baños"])

1    3218
2    1612
3     564
4      56
0       8
5       7
6       1
Name: numero_baños, dtype: int64

In [11]:
df_t.head()

,numero_cuartos,numero_baños,area_de_ construcción_pie2,area_del lote_pie2,numero_pisos,condición,grado,superficie_sótano_pie2,año_de_construcción,precio
0,3,1,1180,5650,1,3,7,0,1955,221900
1,2,1,770,10000,1,3,6,0,1933,180000
2,4,3,1960,5000,1,5,7,910,1965,604000
3,3,2,1680,8080,1,3,8,0,1987,510000
4,3,1,1780,7470,1,3,7,730,1960,229500


# 4. Construcción del modelo
Los algoritmos supervisados en Sckit-learn requieren que las variables de entrada estén separadas de la variable objeto. En este caso, la variable objeto es el precio

In [12]:
#Se selecciona la variable objetivo, en este caso "precio"
Y = df_t['precio']
#Del conjunto de datos se elimina la variable "precio"
X = df_t.drop(['precio'], axis=1)

In [13]:
#Se muestran los datos de Y
Y.head()

0    221900
1    180000
2    604000
3    510000
4    229500
Name: precio, dtype: int64

In [14]:
#Se muestran los datos de X
X.head()

,numero_cuartos,numero_baños,area_de_ construcción_pie2,area_del lote_pie2,numero_pisos,condición,grado,superficie_sótano_pie2,año_de_construcción
0,3,1,1180,5650,1,3,7,0,1955
1,2,1,770,10000,1,3,6,0,1933
2,4,3,1960,5000,1,5,7,910,1965
3,3,2,1680,8080,1,3,8,0,1987
4,3,1,1780,7470,1,3,7,730,1960


Ahora, es necesario hacer la separación de los datos en un conjunto de entrenamiento y otro para realizar la prueba o test. Es así como esta división permite determinar el rendimiento de generalización del modelo sobre nuevos datos.

El conjunto de entranmiento se usa para ajustar (entrenar) un modelo. Posteriormente, se usa el conjunto de test para hacer predicciones, las cuales se comparan con los valores esperados para determinar su rendimiento usando la métrica seleccionada.

1. **Train data:** Se utiliza para entrenar el modelo con el algoritmo de aprendizaje
2. **Test data:** Se utiliza para evaluar el ajusto del modelo

In [15]:
#Se realiza la divisón entrenamiento - test. Se deja el 20% de los datos para el test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2 , random_state=0)

Antes de construir el modelo debemos crear un objeto de la clase **LinearRegression**.

In [16]:
#Primero se crea el objeto para construir el modelo
modelo_regresion = LinearRegression()
#Se comprueba que el modelo fue construido
modelo_regresion

LinearRegression()

A continuación se procede a entrenar el modelo usando el conjunto de entrenamiento.

In [17]:
#Ajustar el modelo con los datos de entrenamiento
modelo_regresion.fit(X_train, Y_train)

LinearRegression()

# 5. Evaluación del modelo
Las métricas de evaluación permitirán determinar qué tan bien se desempeña un modelo, es decir, cómo se ajusta a los datos. En esta sección se utilizan las siguientes métricas:

1. Mean-Squared-Error(MSE). Error medio cuadrático
2. Mean-Absolute-Error(MAE). Error absoluto medio
3. R² o Coeficiente de determinación

In [18]:
#Se obtienen las predicciones del modelo sobre el conjunto de entrenamiento
y_pred = modelo_regresion.predict(X_train)
#Se obtienen las métricas a partir de la predicción y la base de evaluación (valores reales)
print("MSE: %.2f" % mean_squared_error(Y_train, y_pred, squared=True))
print("RMSE: %.2f" % mean_squared_error(Y_train, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_train, y_pred))
print("R²: %.2f" % r2_score(Y_train, y_pred))

MSE: 16003712845.34
RMSE: 126505.78
MAE: 95901.18
R²: 0.50


In [19]:
#Se obtienen las predicciones del modelo sobre el conjuno test.
y_pred = modelo_regresion.predict(X_test)
#Se obtienen las métricas a partir de la predicción y la base de evaluación (valores reales)
print("MSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=True))
print("RMSE: %.2f" % mean_squared_error(Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(Y_test, y_pred))
print("R²: %.2f" % r2_score(Y_test, y_pred))

MSE: 14565133371.50
RMSE: 120686.09
MAE: 95100.90
R²: 0.54


Una vez determinado el rendimiento de generalización usando el conjunto test se puede tomar los datos y construir el modelo final.

In [20]:
#Ajustar el modelo con todos los datos
modelo_regresion.fit(X,Y)

LinearRegression()

In [21]:
#Podemos visualizar los parámetros del modelo (coeficientes de regresión)
modelo_regresion.coef_

array([-1.70410540e+04,  3.20249718e+04,  7.88881222e+01,  1.09289075e-01,
        4.66920658e+04,  1.09592862e+04,  8.91535637e+04,  1.81523616e+01,
       -2.50647984e+03])

In [22]:
#Para visualizar el intercepto, se usa el atributo intercept_ del modelo
modelo_regresion.intercept_

4460731.195425876

# 6. Interpretación del modelo
También podemos sacar un reporte que nos va a permitir analizar estadísticos del modelo

In [23]:
#Ajustar el modelo para ver el reporte
model = sm.OLS(Y,X).fit() ## sm.OLS(output, input)
#Mostrar los estadísticos del modelo
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 precio   R-squared (uncentered):                   0.900
Model:                            OLS   Adj. R-squared (uncentered):              0.900
Method:                 Least Squares   F-statistic:                              5470.
Date:                Tue, 16 Aug 2022   Prob (F-statistic):                        0.00
Time:                        18:17:28   Log-Likelihood:                         -72308.
No. Observations:                5465   AIC:                                  1.446e+05
Df Residuals:                    5456   BIC:                                  1.447e+05
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
numero_cuartos             -1.903e+04   2628.468     -7.239      0.000   -2.42e+04   -1.39e+04
numero_baños               -1530.7478   4190.533     -0.365      0.715   -9745.864    6684.368
area_de_ construcción_pie2    81.6265      5.754     14.185      0.000      70.346      92.908
area_del lote_pie2             0.0791      0.042      1.906      0.057      -0.002       0.161
numero_pisos                4.313e+04   5445.626      7.919      0.000    3.24e+04    5.38e+04
condición                   2.538e+04   2641.505      9.607      0.000    2.02e+04    3.06e+04
grado                       7.581e+04   2916.489     25.995      0.000    7.01e+04    8.15e+04
superficie_sótano_pie2        47.1671      6.752      6.985      0.000      33.930      60.404
año_de_construcción         -179.2292     10.487    -17.090      0.000    -199.789    -158.670
==============================================================================
Omnibus:                      875.262   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2140.377
Skew:                           0.900   Prob(JB):                         0.00
Kurtosis:                       5.481   Cond. No.                     1.48e+05
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 1.48e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# 7. Almacenamiento del modelo.
Puedes guardar tu modelo para que lo puedas recuperar caundo lo desees o llevarlo a producción.

In [24]:
#Una forma de guardar el modelo es a través de la funcionalidad joblib
import joblib

In [25]:
#Ahora se guarda el modelo con el nombre que se desee.
joblib.dump(modelo_regresion, 'ModeloRegresion.joblib')
#Si se desea recuperar el modelo y cargarlo se usa la siguiente instrucción:
#modelo = load('ModeloRegresion.joblib')

['ModeloRegresion.joblib']

# 8. Construcción del modelo - Regresión polinomial

In [26]:
df_p = df_t

In [27]:
#Se selecciona la variable objetivo, en este caso "precio"
Y_p = df_p['precio']
#Del conjunto de datos se elimina la variable "precio"
X_p = df_p.drop(['precio'], axis=1)

Ahora vamos a aplicar la transformación polinomial a las variables de entrada. Como primer paso creamos un objeto de la clase PolynomicalFeatures de sklearn. Luego ajustamos y transformamos los datos.

In [28]:
#Utilizaremos una transformada de grado 2.
poly = PolynomialFeatures(degree=2)
poly_X = poly.fit_transform(X_p)

In [29]:
poly_X.shape

(5465, 55)

Ejemplo:

En caso de tener dos variables explicativas [a,b], el resultado de la transformación polinomial con grado 2 sería el siguiente [1, a, b, a^2, a*b, b^2] 

In [30]:
#Se realiza la división de entrenamiento - test. Se deja el 20% de los datos para el test:
poly_X_train, poly_X_test, poly_Y_train, poly_Y_test = train_test_split(poly_X, Y_p, test_size = 0.2, random_state = 0)

In [31]:
#Creación del objeto de la clase LinearRegression y ajusto del modelo de los datos.
modelo_regresion_poly = LinearRegression()
#Podemos verificar que lo hemos construido
modelo_regresion_poly

LinearRegression()

In [32]:
#Ajustar el modelo con los datos de entrenamiento con las nuevas variables polinomiales
modelo_regresion_poly.fit(poly_X_train, poly_Y_train)

LinearRegression()

# 9. Evaluación del modelo polinomial

In [33]:
#Se obtienen las predicciones del modelo sobre el conjunto test.
y_pred = modelo_regresion_poly.predict(poly_X_test)

In [34]:
##Se obtienen las métricas a partir de la predicción y la base de evaluación (valores reales)
print("MSE: %.2f" % mean_squared_error(poly_Y_test, y_pred, squared=True))
print("RMSE: %.2f" % mean_squared_error(poly_Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(poly_Y_test, y_pred))
print("R²: %.2f" % r2_score(Y_test, y_pred))

MSE: 15714038571.01
RMSE: 125355.65
MAE: 94672.13
R²: 0.50


# 10. Regresión regularizada - Caso Ridge
Para  realizar este tipo de regularización se siguen los siguientes pasos:
1. Preparar los datos del modelo
2. Utilizar técnicas de validación para el ajuste de hiperparámetros
3. Construir el modelo de regresión Ridge
4. Evaluar el modelo

In [35]:
df_R = df_t

In [36]:
#Se selecciona la variable objetivo, en este caso "precio"
Y_R = df_R['precio']
#Del conjunto de datos se elimina la variable "precio"
X_R = df_R.drop(['precio'], axis=1)

In [37]:
#Se realiza la división de entrenamiento - test. Se deja el 20% de los datos para el test:
ridge_X_train, ridge_X_test, ridge_Y_train, ridge_Y_test = train_test_split(X_R, Y_R, test_size = 0.2, random_state = 0)

La regresión Ridge usa la norma L2 del vector de coeficientes como término de regularización. Para controlar la cantidad de regularización se usa el hiperparámetro alpha. Se prueba con un valor igual a 2.

In [38]:
modelo_ridge = Ridge(alpha=2, normalize = True)
modelo_ridge

Ridge(alpha=2, normalize=True)

In [39]:
#Ajuste del modelo
modelo_ridge.fit(ridge_X_train, ridge_Y_train)

C:\Users\felip\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(


Ridge(alpha=2, normalize=True)

In [40]:
#Ahora se pruba el rendimiento sobre el conjunto test.
y_pred = modelo_ridge.predict(ridge_X_test)
print("RMSE: %.2f" % mean_squared_error(ridge_Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(ridge_Y_test, y_pred))
print("R²: %.2f" % r2_score(ridge_Y_test, y_pred))

RMSE: 140219.24
MAE: 113244.91
R²: 0.37


¿Cómo podemos saber el mejor valor de alpha? Este hiperparámetro se debe ajustar y no es posible usar el conjunto test para esta tarea. Es por esto que se debe recurrrir a las técnicas de validación las cuales, de forma general, se pueden implementar de dos maneras: usando un conjunto de validación o empleando la validación cruzada de k particiones (K-fold validation)

a) **Con un conjunto de validación .** Usando esta forma se debe dividir el conjunto de entrenamiento en dos: Uno para la construcción del modelo (se toma como el nuevo conjunto de entrenamiento) y el otro para la validación (para determinar los valores de los hiperparámetros y complejidad)

Veamos a continuación cómo hacerlo:

In [41]:
#Se divide el conjunto de entrenamiento en dos: Una para la construcción del modelo y otro para la validación, el cual será
#usado para determinar el rendimiento del modelo con una combinación específica de hiperparámetros.
X_trainval, X_val, Y_trainval, Y_val = train_test_split(ridge_X_train, ridge_Y_train, test_size = 0.15, random_state = 0)

In [42]:
#Se fija un valor de alpha en 1, se construye el modelo y se prueba sobre la validación. Se utiliza MAE como métrica de 
#rendimiento para hacer la selección.
modelo_ridge = Ridge(alpha=1, normalize = True)
modelo_ridge.fit(X_trainval, Y_trainval)
y_pred = modelo_ridge.predict(X_val)
print("MAE: %.2f" % mean_absolute_error(Y_val, y_pred))

MAE: 104575.44


C:\Users\felip\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(


In [43]:
#Se fija un valor de alpha en 2, se construye el modelo y se prueba sobre la validación
modelo_ridge = Ridge(alpha=2, normalize = True)
modelo_ridge.fit(X_trainval, Y_trainval)
y_pred = modelo_ridge.predict(X_val)
print("MAE: %.2f" % mean_absolute_error(Y_val, y_pred))

MAE: 110110.47


C:\Users\felip\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(


In [44]:
#Se fija un valor de alpha en 5, se construye el modelo y se prueba sobre la validación
modelo_ridge = Ridge(alpha=5, normalize = True)
modelo_ridge.fit(X_trainval, Y_trainval)
y_pred = modelo_ridge.predict(X_val)
print("MAE: %.2f" % mean_absolute_error(Y_val, y_pred))

MAE: 118796.75


C:\Users\felip\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(


El mejor alpha, según **MAE** es 1. Ahora, se selecciona este modelo como el final (por esta razón también estas técnicas se conocen como métodos de selección de modelos) y determinamos su rendimiento de generalización. Se tiene en cuenta que ahora se debe usar todo el conjunto de entrenamiento.

In [45]:
modelo_ridge = Ridge(alpha=1, normalize = True)
modelo_ridge.fit(ridge_X_train, ridge_Y_train)
y_pred = modelo_ridge.predict(ridge_X_test)
print("RMSE: %.2f" % mean_squared_error(ridge_Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(ridge_Y_test, y_pred))
print("R²: %.2f" % r2_score(ridge_Y_test, y_pred))

RMSE: 131995.43
MAE: 106796.74
R²: 0.44


C:\Users\felip\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(


Estos pasos de asignar un alpha, entrenar y probar con validación pueden ser implementados en un ciclo.

Ahora usamos la otra opción:

b) **Validación cruzada de k-particiones.** En este caos el conjunto de entrenamiento se divide en en k particiones disjuntas. Luego, se fija un valor de alpha y se toma como conjunto de validación la primera partición y el resto para entrenar el modelo. Este proceso se repite para cada partición. Al temrinar, se determina el valor promedio de las métricas seleccionadas. Este ciclo se repite sobre diferentes valores de alpha y se selecciona aquel que ofrezca el mejor rendimiento.

**Sckit-learn** ofrece algunos métodos que automatizan el proceso de busque de los valores de los hiperparámetros. Uno de ellos es **GridSearchCV**, el cual se basa en la validación cruzada de k-particiones. A continuación, se muestra cómo usarlo.

In [46]:
#Se fija el número de particiones. Se usa K = 10.
particiones = KFold(n_splits=10, shuffle=True, random_state = 0)

In [47]:
# from xgboost import XGBRegressor
# xgb_model = XGBRegressor(random_state = 0)

In [48]:
#Ahora se debe fijar el espacio de búsqueda, es decir, lso valores de alpha que se quieren que sean considerados. Para esto
#se define un diccionario (o grilla) con los valores que podrá asumir el hiperparámetro alpha.
#Sr prueban con los siguientes valores:
param_grid = {'alpha': [0.1,0.25,0.5]}

In [49]:
#Se define el modelo sin ningún valor del hiperparámetro alpha
modelo_ridge = Ridge(normalize = True)

In [50]:
#Ahora se usa GridSearch sobre el grid definido y con 10 pariciones en la validación cruzada
mejor_modelo = GridSearchCV(modelo_ridge, param_grid, cv=particiones, n_jobs=-1)
#Se ajusta el modelo
mejor_modelo.fit(ridge_X_train, ridge_Y_train)

C:\Users\felip\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(


GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             estimator=Ridge(normalize=True), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.25, 0.5]})

In [51]:
#Para observar cuál fue el resultado de la búsqueda (mejor valor de alpha)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_))

Mejor parámetro: {'alpha': 0.1}


In [52]:
#También es posible indicarle a GridSearch que seleccione el mejor modelo a partir de la búsqueda con base en una métrica 
#particular. Por ejemplo, hubiera sido posible usar la siguient línea de comando:
mejor_modelo = GridSearchCV(modelo_ridge, param_grid, scoring = 'neg_mean_squared_error', n_jobs=-1)
mejor_modelo.fit(ridge_X_train, ridge_Y_train)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_))

Mejor parámetro: {'alpha': 0.1}


C:\Users\felip\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Ridge())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * n_samples. 
  warnings.warn(


Un aspecto importante que se debe de tener en cuenta es que **GridSearchCV** devuelve el modelo entrenado con el mejor valor del hiperparámetro alpha (resultado de la búsqueda).

In [53]:
modelo_final = mejor_modelo.best_estimator_
#Se prueba ahora este modelo sobre el test
y_pred = modelo_final.predict(ridge_X_test)
print("RMSE: %.2f" % mean_squared_error(ridge_Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(ridge_Y_test, y_pred))
print("R²: %.2f" % r2_score(ridge_Y_test, y_pred))

RMSE: 120523.88
MAE: 95700.59
R²: 0.54


# 10. Regresión regularizada - Caso Lasso
Para  realizar este tipo de regularización con norma L1 se siguen los siguientes pasos:
1. Preparar los datos del modelo
2. Utilizar técnicas de validación para el ajuste de hiperparámetros
3. Construir el modelo de regresión Lasso
4. Evaluar el modelo
5. Determinar importancia de variables

In [54]:
df_l = df_t

In [55]:
#Se selecciona la variable objetivo, en este caso "precio"
Y_L = df_R['precio']
#Del conjunto de datos se elimina la variable "precio"
X_L = df_R.drop(['precio'], axis=1)

In [56]:
#Se realiza la división de entrenamiento - test. Se deja el 20% de los datos para el test:
lasso_X_train, lasso_X_test, lasso_Y_train, lasso_Y_test = train_test_split(X_L, Y_L, test_size = 0.2, random_state = 0)

Tal y como se sabe, la regresión Lasso usa la norma L1 del vector de coeficientes como término de regularización. Para controlar la cantidad de regularización se usa el hiperparámetro alpha. Se prueba con un valor de 1.

In [57]:
# Para entender la convergencia del algoritmo que se usa en Lasso para optimizar la función de costo, se usa la opción de 
# normalizar los datos para que todos estén en el mismo rango.
modelo_lasso = Lasso(alpha=1, normalize = True)
modelo_lasso

Lasso(alpha=1, normalize=True)

In [58]:
#Se ajusta el modelo
modelo_lasso.fit(lasso_X_train, lasso_Y_train)

C:\Users\felip\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


Lasso(alpha=1, normalize=True)

In [59]:
# Se prueba el rendimiento sobre el conjunto test.


In [60]:
y_pred = modelo_lasso.predict(lasso_X_test)
print("RMSE: %.2f" % mean_squared_error(lasso_Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(lasso_Y_test, y_pred))
print("R²: %.2f" % r2_score(lasso_Y_test, y_pred))

RMSE: 120686.11
MAE: 95115.78
R²: 0.54


Ahora se usa la función **GridSearchCV** para la búsqueda del valor alpha.

In [61]:
#Se fija el número de particiones. Se usa K = 10.
particiones = KFold(n_splits=10, shuffle=True, random_state = 0)

In [62]:
#Ahora se debe fijar el espacio de búsqueda, es decir, lso valores de alpha que se quieren que sean considerados. Para esto
#se define un diccionario (o grilla) con los valores que podrá asumir el hiperparámetro alpha.
#Sr prueban con los siguientes valores:
param_grid = {'alpha': [1, 2, 3, 4, 5]}

In [63]:
#Definimos el modelo sin ningún valor del hiperparámetro alpha
modelo_lasso = Lasso(normalize = True)

In [64]:
#Ahora se usa GridSearch sobre el grid definido y con 10 pariciones en la validación cruzada
mejor_modelo = GridSearchCV(modelo_lasso, param_grid, cv=particiones, n_jobs=-1)
#Se ajusta el modelo
mejor_modelo.fit(lasso_X_train, lasso_Y_train)

C:\Users\felip\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             estimator=Lasso(normalize=True), n_jobs=-1,
             param_grid={'alpha': [1, 2, 3, 4, 5]})

In [65]:
#Para observar cuál fue el resultado de la búsqueda (mejor valor de alpha)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_))

Mejor parámetro: {'alpha': 2}


In [66]:
#También es posible indicarle a GridSearch que seleccione el mejor modelo a partir de la búsqueda con base en una métrica 
#particular. Por ejemplo, hubiera sido posible usar la siguient línea de comando:
mejor_modelo = GridSearchCV(modelo_lasso, param_grid, scoring = 'neg_mean_squared_error', n_jobs=-1)
mejor_modelo.fit(lasso_X_train, lasso_Y_train)
print("Mejor parámetro: {}".format(mejor_modelo.best_params_))

Mejor parámetro: {'alpha': 1}


C:\Users\felip\anaconda3\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


Un aspecto importante que se debe de tener en cuenta es que **GridSearchCV** devuelve el modelo entrenado con el mejor valor del hiperparámetro alpha (resultado de la búsqueda).

In [67]:
modelo_final = mejor_modelo.best_estimator_
#Se prueba ahora este modelo sobre el test
y_pred = modelo_final.predict(lasso_X_test)
print("RMSE: %.2f" % mean_squared_error(lasso_Y_test, y_pred, squared=False))
print("MAE: %.2f" % mean_absolute_error(lasso_Y_test, y_pred))
print("R²: %.2f" % r2_score(lasso_Y_test, y_pred))

RMSE: 120686.11
MAE: 95115.78
R²: 0.54


# Importancia de variables y selección
La regresión Lasso actúa como un selector de variables: Será aquellas que tengan un coeficientes diferente de cero. Así, con base de los valores de los coeficientes, se determina la importancia de los atributos en el análisis y predicción

In [68]:
# Revisar los parámetros del modelo entrenado
coeficientes = modelo_final.coef_
variables = lasso_X_train.columns
# Se muestra en una tabla los valores de los coeficientes para cada variable
pd.DataFrame({'coeficientes':coeficientes, 'variables':variables})

,coeficientes,variables
0,-16311.745215,numero_cuartos
1,31524.149606,numero_baños
2,80.208018,area_de_ construcción_pie2
3,0.075234,area_del lote_pie2
4,44571.677845,numero_pisos
5,10707.089716,condición
6,91076.065562,grado
7,13.448970,superficie_sótano_pie2
8,-2576.574166,año_de_construcción
